In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [2]:
X_dataset = 'train_X1.csv'
y_dataset = 'train_y1.csv'
model_save_path = 'model/model.hdf5'
tflite_save_path = 'model/model.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 4

# Dataset reading

In [177]:
X_raw = np.loadtxt(X_dataset, delimiter=',', dtype='float32')
y_raw = np.loadtxt(y_dataset, delimiter=',', dtype='str')

In [156]:
X_raw = np.loadtxt(X_dataset, delimiter=',')
y_raw = np.loadtxt(y_dataset, delimiter=',', dtype='str')

In [178]:
# tmp fix
y_raw_fix = y_raw[:]
y_raw_fix = np.where(y_raw_fix=='hit', 0, y_raw_fix)
y_raw_fix = np.where(y_raw_fix=='stand', 1, y_raw_fix)
y_raw_fix = np.where(y_raw_fix=='split', 2, y_raw_fix)
y_raw_fix = np.where(y_raw_fix=='reset', 3, y_raw_fix)
y_raw_fix = y_raw_fix.astype(int)
y_raw_fix

array([0, 0, 0, ..., 3, 3, 3])

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw_fix, train_size=0.8, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.75, random_state=RANDOM_SEED)

In [180]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(970, 66)
(324, 66)
(324, 66)


# Model building

In [181]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((66, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_6 (Dropout)          (None, 66)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 20)                1340      
_________________________________________________________________
dropout_7 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 44        
Total params: 1,594
Trainable params: 1,594
Non-trainable params: 0
_________________________________________________________________


In [182]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [183]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Load Model

In [ ]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

# Model training

In [184]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_val, y_val),
    callbacks=[cp_callback, es_callback]
)

Train on 970 samples, validate on 324 samples
Epoch 1/1000
128/970 [==>...........................] - ETA: 0s - loss: 1.7036 - accuracy: 0.1875
Epoch 00001: saving model to model/model.hdf5
970/970 [==============================] - 0s 196us/sample - loss: 1.5864 - accuracy: 0.2227 - val_loss: 1.4181 - val_accuracy: 0.1235
Epoch 2/1000
128/970 [==>...........................] - ETA: 0s - loss: 1.5171 - accuracy: 0.1875
Epoch 00002: saving model to model/model.hdf5
970/970 [==============================] - 0s 23us/sample - loss: 1.4591 - accuracy: 0.2289 - val_loss: 1.3992 - val_accuracy: 0.0463
Epoch 3/1000
128/970 [==>...........................] - ETA: 0s - loss: 1.4562 - accuracy: 0.1875
Epoch 00003: saving model to model/model.hdf5
970/970 [==============================] - 0s 22us/sample - loss: 1.4149 - accuracy: 0.2361 - val_loss: 1.3904 - val_accuracy: 0.1296
Epoch 4/1000
128/970 [==>...........................] - ETA: 0s - loss: 1.3785 - accuracy: 0.2266
Epoch 00004: saving m

128/970 [==>...........................] - ETA: 0s - loss: 0.7259 - accuracy: 0.7188
Epoch 00030: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.7190 - accuracy: 0.7237 - val_loss: 0.4017 - val_accuracy: 0.9877
Epoch 31/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.7336 - accuracy: 0.7188
Epoch 00031: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.7250 - accuracy: 0.7216 - val_loss: 0.3863 - val_accuracy: 0.9815
Epoch 32/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.6636 - accuracy: 0.7734
Epoch 00032: saving model to model/model.hdf5
970/970 [==============================] - 0s 25us/sample - loss: 0.6725 - accuracy: 0.7567 - val_loss: 0.3631 - val_accuracy: 0.9877
Epoch 33/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.6199 - accuracy: 0.7969
Epoch 00033: saving model to model/model.hdf5
970/970 [=======================

Epoch 00059: saving model to model/model.hdf5
970/970 [==============================] - 0s 25us/sample - loss: 0.3993 - accuracy: 0.8495 - val_loss: 0.0968 - val_accuracy: 0.9969
Epoch 60/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.3947 - accuracy: 0.8828
Epoch 00060: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.3638 - accuracy: 0.8649 - val_loss: 0.0936 - val_accuracy: 0.9969
Epoch 61/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.3734 - accuracy: 0.8750
Epoch 00061: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.3649 - accuracy: 0.8753 - val_loss: 0.0940 - val_accuracy: 0.9969
Epoch 62/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.3743 - accuracy: 0.8906
Epoch 00062: saving model to model/model.hdf5
970/970 [==============================] - 0s 23us/sample - loss: 0.3615 - accuracy: 0.8753 - val_loss: 0.0931 - val_

Epoch 89/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2520 - accuracy: 0.9219
Epoch 00089: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.2739 - accuracy: 0.9041 - val_loss: 0.0465 - val_accuracy: 1.0000
Epoch 90/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1962 - accuracy: 0.9219
Epoch 00090: saving model to model/model.hdf5
970/970 [==============================] - 0s 25us/sample - loss: 0.2760 - accuracy: 0.9031 - val_loss: 0.0459 - val_accuracy: 1.0000
Epoch 91/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2070 - accuracy: 0.9453
Epoch 00091: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.2709 - accuracy: 0.9062 - val_loss: 0.0440 - val_accuracy: 1.0000
Epoch 92/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2489 - accuracy: 0.8984
Epoch 00092: saving model to model/model.hdf5
970/970 [=========

128/970 [==>...........................] - ETA: 0s - loss: 0.1933 - accuracy: 0.9219
Epoch 00118: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.2084 - accuracy: 0.9278 - val_loss: 0.0280 - val_accuracy: 1.0000
Epoch 119/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2904 - accuracy: 0.9062
Epoch 00119: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.2302 - accuracy: 0.9165 - val_loss: 0.0288 - val_accuracy: 1.0000
Epoch 120/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2246 - accuracy: 0.9219
Epoch 00120: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.2220 - accuracy: 0.9247 - val_loss: 0.0295 - val_accuracy: 1.0000
Epoch 121/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1878 - accuracy: 0.9297
Epoch 00121: saving model to model/model.hdf5
970/970 [====================

128/970 [==>...........................] - ETA: 0s - loss: 0.1907 - accuracy: 0.9688
Epoch 00147: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.2019 - accuracy: 0.9330 - val_loss: 0.0226 - val_accuracy: 1.0000
Epoch 148/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1503 - accuracy: 0.9453
Epoch 00148: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1890 - accuracy: 0.9351 - val_loss: 0.0206 - val_accuracy: 0.9969
Epoch 149/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1865 - accuracy: 0.9375
Epoch 00149: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.2205 - accuracy: 0.9216 - val_loss: 0.0206 - val_accuracy: 0.9969
Epoch 150/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1658 - accuracy: 0.9453
Epoch 00150: saving model to model/model.hdf5
970/970 [====================

128/970 [==>...........................] - ETA: 0s - loss: 0.2177 - accuracy: 0.9453
Epoch 00176: saving model to model/model.hdf5
970/970 [==============================] - 0s 23us/sample - loss: 0.1888 - accuracy: 0.9495 - val_loss: 0.0189 - val_accuracy: 1.0000
Epoch 177/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1235 - accuracy: 0.9609
Epoch 00177: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1890 - accuracy: 0.9371 - val_loss: 0.0187 - val_accuracy: 1.0000
Epoch 178/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1501 - accuracy: 0.9297
Epoch 00178: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1974 - accuracy: 0.9278 - val_loss: 0.0198 - val_accuracy: 1.0000
Epoch 179/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1854 - accuracy: 0.9375
Epoch 00179: saving model to model/model.hdf5
970/970 [====================

128/970 [==>...........................] - ETA: 0s - loss: 0.0960 - accuracy: 0.9844
Epoch 00205: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1918 - accuracy: 0.9392 - val_loss: 0.0155 - val_accuracy: 1.0000
Epoch 206/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1842 - accuracy: 0.9219
Epoch 00206: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1792 - accuracy: 0.9423 - val_loss: 0.0159 - val_accuracy: 0.9969
Epoch 207/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2190 - accuracy: 0.9219
Epoch 00207: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1992 - accuracy: 0.9299 - val_loss: 0.0164 - val_accuracy: 1.0000
Epoch 208/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2386 - accuracy: 0.9297
Epoch 00208: saving model to model/model.hdf5
970/970 [====================

128/970 [==>...........................] - ETA: 0s - loss: 0.1579 - accuracy: 0.9688
Epoch 00234: saving model to model/model.hdf5
970/970 [==============================] - 0s 23us/sample - loss: 0.1619 - accuracy: 0.9474 - val_loss: 0.0115 - val_accuracy: 1.0000
Epoch 235/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1697 - accuracy: 0.9453
Epoch 00235: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1336 - accuracy: 0.9536 - val_loss: 0.0110 - val_accuracy: 1.0000
Epoch 236/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.2010 - accuracy: 0.9297
Epoch 00236: saving model to model/model.hdf5
970/970 [==============================] - 0s 24us/sample - loss: 0.1592 - accuracy: 0.9505 - val_loss: 0.0115 - val_accuracy: 0.9969
Epoch 237/1000
128/970 [==>...........................] - ETA: 0s - loss: 0.1792 - accuracy: 0.9453
Epoch 00237: saving model to model/model.hdf5
970/970 [====================

In [185]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

324/324 [==============================] - 0s 9us/sample - loss: 0.0370 - accuracy: 0.9877


In [189]:
%%time
# Inference test
pred_idx = 6

predict_result = model.predict(np.array([X_test[pred_idx]]))
print(np.squeeze(predict_result))
print(f"Pred: {np.argmax(np.squeeze(predict_result))}")
print(f"ANS: {y_test[pred_idx]}")

[4.6442026e-05 1.5373927e-05 2.5377291e-05 9.9991286e-01]
Pred: 3
ANS: 3
Wall time: 13 ms


In [190]:
# Save model
model.save(model_save_path, include_optimizer=False)

# Convert to model for Tensorflow-Lite

In [191]:
# Transform model (quantization)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open(tflite_save_path, 'wb') as f:
    f.write(tflite_model)

# Inference test

In [192]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [193]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [204]:
np.array([X_test[pred_idx]])

array([[0.9251988 , 0.51947194, 0.27901644, 0.13589287, 0.        ,
        0.4903345 , 0.37482685, 0.33350947, 0.29810655, 0.68663096,
        0.5830309 , 0.5356928 , 0.50649995, 0.85322547, 0.8160409 ,
        0.80324686, 0.7915903 , 1.        , 0.99305075, 0.99061704,
        0.9867591 , 1.        , 0.97216254, 0.7941084 , 0.6198479 ,
        0.52388275, 0.5372412 , 0.29367837, 0.18010232, 0.0881668 ,
        0.4693069 , 0.2279393 , 0.09990688, 0.01362208, 0.47032097,
        0.23938008, 0.1034162 , 0.        , 0.51399857, 0.33277565,
        0.2325447 , 0.14466786, 0.        , 0.08102668, 0.24253784,
        0.40305084, 0.57805365, 0.4907771 , 0.77218354, 0.9209106 ,
        1.        , 0.53964156, 0.8362205 , 0.92907786, 0.9696973 ,
        0.57644224, 0.8134738 , 0.85235834, 0.8474466 , 0.61670136,
        0.81893307, 0.8920385 , 0.9205032 , 0.36626148, 0.5795952 ,
        0.21629396]], dtype=float32)

In [202]:
pred_idx = 12
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[pred_idx]]))

In [196]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

Wall time: 0 ns


In [197]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))
print(f"ANS: {y_test[pred_idx]}")

[2.1436722e-04 9.9977654e-01 3.8282751e-06 5.1896882e-06]
1
ANS: 1
